In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import spectrogram
import tensorflow as tf
import random
import keras
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout

import librosa

2024-06-04 11:37:01.416021: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 11:37:01.417337: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 11:37:01.476796: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 11:37:01.714823: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 11:37:02.752176: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
fileList = {}
emotionClass = {
    "Happy": 0,
    "Angry": 1,
    "Suprised": 2,
    "Sad": 3,
    "Fearful": 4,
    "Neutral": 5,
    "Disgusted": 6,
}
# KAGGLE ===================
# BASE_PATH = '/kaggle/input/audio-emotions/Emotions/'
# for emotion in os.listdir(BASE_PATH):
#     fileList[emotion] = []
#     for file in os.listdir(f"{BASE_PATH}/{emotion}"):
#         fileList[emotion].append(f"{BASE_PATH}{emotion}/{file}")

for emotion in os.listdir("./archive/Emotions"):
    fileList[emotion] = []
    counter = 0
    for file in os.listdir(f"{"./archive/Emotions"}/{emotion}"):
        fileList[emotion].append(f"{"./archive/Emotions/"}{emotion}/{file}")
        counter += 1
        if counter > 40: break

In [ ]:
for k, v in fileList.items():
    print(f"{k:10s} ==> {len(v)}")

# Lee el archivo WAV
sample_rate, data = wavfile.read(fileList["Neutral"][5])

In [88]:
train = []
max_length = -1
mfcc_number = 15

# Calcula el espectrograma
for emotion, filePath in fileList.items():
    for file in filePath:
        sample_rate, data = wavfile.read(file)

        # y, sr   = librosa.load(file)
        # mfcc    = librosa.feature.mfcc(y=y, n_mfcc=mfcc_number, sr=sr)

        frequencies, times, spectrogram_data = spectrogram(data, fs=sample_rate)
        spectrogram_data = spectrogram_data.T

        train.append(
            (
                spectrogram_data.T,
                # mfcc.T,
                [emotionClass[emotion]],
            )
        )

        if np.shape(mfcc.T)[0] > max_length:
            max_length = np.shape(mfcc.T)[0]

X_train = list(list(zip(*train))[0])
Y_train = list(list(zip(*train))[1])
print(Y_train)
# Y_train = list(map(lambda a: to_categorical([a], num_classes=7), Y_train))
Y_train = to_categorical(Y_train, num_classes=7)

for index, element in enumerate(X_train):
    original_shape = np.shape(element)[0]
    array_zeros = np.zeros((max_length, mfcc_number))
    array_zeros[:original_shape, :] = element
    X_train[index] = array_zeros

    # array_tag = to_categorical(np.array([Y_train[index]]), num_classes=7)
    # array_tag = np.repeat(array_tag, original_shape, axis=0)    
    # array_zeros = np.zeros((max_length, 7))
    # array_zeros[:original_shape, :] = array_tag

    # Y_train[index] = array_zeros  
# print(np.shape(Y_train))

np.concatenate(X_train, axis=0)
X_train = np.array(X_train)
# np.concatenate(Y_train, axis=0)

print(type(X_train))


[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [5], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6], [6]]
<class 'numpy.ndarray'>


In [96]:
# Definir el modelo
model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(None, mfcc_number)))
model.add(LSTM(128, input_shape=(None)))
# model.add(Dropout(0.3))ç
model.add(Dense(32, activation="relu"))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Mostrar resumen del modelo
# model.summary()

model.fit(X_train, Y_train, epochs=20, batch_size=32, shuffle=True)

Epoch 1/20


/home/aaron/miniconda/envs/audio/lib/python3.12/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.1847 - loss: 1.9802
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.1506 - loss: 1.9234
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.2548 - loss: 1.8570
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.2246 - loss: 1.8588
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.2261 - loss: 1.8365
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.2150 - loss: 1.8057
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.2848 - loss: 1.7749
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.3101 - loss: 1.7230
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.2908 - loss: 1.7352
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.3699 - loss: 1.6763
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.4098 - loss: 1.6575
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.4480 - loss: 1.6442
Epoch 13/20
